In [1]:
from google.colab import drive
drive.mount('/content/drive')

import pandas as pd
import numpy as np
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

Mounted at /content/drive


In [2]:
project_folder = '/content/drive/MyDrive/Healthcare_Analysis_Project'
data_raw_path = f'{project_folder}/data_raw'
data_processed_path = f'{project_folder}/data_processed'


Clean CDC Social Vulnerability Index Data


In [3]:
def clean_cdc_svi():
    """
    Clean and transform CDC SVI data for analysis
    """
    print("="*70)
    print("CLEANING CDC SOCIAL VULNERABILITY INDEX DATA")
    print("="*70)

    # Load raw data
    print("\n Loading raw data...")
    df = pd.read_csv(f"{data_raw_path}/cdc_svi_raw.csv")
    print(f"✓ Loaded {len(df):,} raw records")

    # Select relevant columns
    print("\n Selecting relevant columns...")
    columns_to_keep = [
        'FIPS',          # County FIPS code
        'COUNTY',        # County name
        'STATE',         # State abbreviation
        'ST_ABBR',       # State abbreviation (backup)
        'LOCATION',      # Full location name
        'E_TOTPOP',      # Total population estimate
        'EP_POV150',     # % persons below 150% poverty
        'EP_UNEMP',      # % unemployment
        'EP_NOHSDP',     # % no high school diploma
        'EP_UNINSUR',    # % uninsured
        'EP_AGE65',      # % aged 65+
        'EP_AGE17',      # % aged 17 and younger
        'EP_DISABL',     # % civilian with a disability
        'EP_SNGPNT',     # % single-parent households
        'RPL_THEMES'     # Overall percentile ranking
    ]

    # Keep only columns that exist
    available_cols = [col for col in columns_to_keep if col in df.columns]
    df_clean = df[available_cols].copy()
    print(f"✓ Selected {len(available_cols)} columns")

    # Rename for clarity
    print("\n  Renaming columns...")
    rename_dict = {
        'FIPS': 'fips',
        'COUNTY': 'county',
        'STATE': 'state',
        'ST_ABBR': 'state_abbr',
        'LOCATION': 'location',
        'E_TOTPOP': 'total_population',
        'EP_POV150': 'poverty_rate',
        'EP_UNEMP': 'unemployment_rate',
        'EP_NOHSDP': 'no_hs_diploma_rate',
        'EP_UNINSUR': 'uninsured_rate',
        'EP_AGE65': 'seniors_pct',
        'EP_AGE17': 'youth_pct',
        'EP_DISABL': 'disability_rate',
        'EP_SNGPNT': 'single_parent_pct',
        'RPL_THEMES': 'svi_percentile'
    }

    # Rename only columns that exist
    rename_dict = {k: v for k, v in rename_dict.items() if k in df_clean.columns}
    df_clean = df_clean.rename(columns=rename_dict)

    # Handle state column
    if 'state' not in df_clean.columns and 'state_abbr' in df_clean.columns:
        df_clean['state'] = df_clean['state_abbr']

    print("✓ Columns renamed")

    # Data quality checks
    print("\n Data Quality Checks:")
    print(f"   • Missing FIPS codes: {df_clean['fips'].isnull().sum():,}")
    print(f"   • Duplicate FIPS codes: {df_clean['fips'].duplicated().sum():,}")

    # Remove rows with missing FIPS
    df_clean = df_clean.dropna(subset=['fips'])
    print(f"✓ Removed records without FIPS codes")

    # Convert -999 (CDC's missing value indicator) to NaN
    print("\n Handling missing values...")
    df_clean = df_clean.replace(-999, np.nan)
    df_clean = df_clean.replace(-999.0, np.nan)

    # For percentage columns, fill NaN with 0
    percentage_cols = [col for col in df_clean.columns if
                      any(x in col for x in ['rate', 'pct', 'percentile'])]

    if percentage_cols:
        df_clean[percentage_cols] = df_clean[percentage_cols].fillna(0)
        print(f"✓ Filled missing values in {len(percentage_cols)} percentage columns")

    # Ensure FIPS is string format with leading zeros
    df_clean['fips'] = df_clean['fips'].astype(str).str.zfill(5)

    # Save cleaned data
    output_path = f"{data_processed_path}/county_vulnerability_clean.csv"
    df_clean.to_csv(output_path, index=False)

    print(f"\n CLEANING COMPLETE!")
    print(f" Saved: {output_path}")
    print(f" Final dataset: {len(df_clean):,} counties, {len(df_clean.columns)} columns")

    return df_clean

# Run cleaning function
vulnerability_df = clean_cdc_svi()

# Display preview and summary
print("\n" + "="*70)
print("DATA PREVIEW")
print("="*70)
display(vulnerability_df.head(10))

print("\n" + "="*70)
print("SUMMARY STATISTICS")
print("="*70)
display(vulnerability_df.describe())

CLEANING CDC SOCIAL VULNERABILITY INDEX DATA

 Loading raw data...
✓ Loaded 2,000 raw records

 Selecting relevant columns...
✓ Selected 15 columns

  Renaming columns...
✓ Columns renamed

 Data Quality Checks:
   • Missing FIPS codes: 0
   • Duplicate FIPS codes: 0
✓ Removed records without FIPS codes

 Handling missing values...
✓ Filled missing values in 9 percentage columns

 CLEANING COMPLETE!
 Saved: /content/drive/MyDrive/Healthcare_Analysis_Project/data_processed/county_vulnerability_clean.csv
 Final dataset: 2,000 counties, 15 columns

DATA PREVIEW


,fips,county,state,state_abbr,location,total_population,poverty_rate,unemployment_rate,no_hs_diploma_rate,uninsured_rate,seniors_pct,youth_pct,disability_rate,single_parent_pct,svi_percentile
0,1001020100,Autauga County,Alabama,AL,Census Tract 201; Autauga County; Alabama,1865,21.6,2.6,16.2,8.2,19.5,24.0,20.1,4.9,0.3635
1,1001020200,Autauga County,Alabama,AL,Census Tract 202; Autauga County; Alabama,1861,14.3,5.5,8.7,7.8,15.7,9.9,20.4,0.7,0.4155
2,1001020300,Autauga County,Alabama,AL,Census Tract 203; Autauga County; Alabama,3492,22.2,1.9,10.6,5.1,13.5,26.2,16.0,8.5,0.4843
3,1001020400,Autauga County,Alabama,AL,Census Tract 204; Autauga County; Alabama,3987,12.9,0.9,7.3,4.4,26.0,14.6,22.3,3.7,0.2386
4,1001020501,Autauga County,Alabama,AL,Census Tract 205.01; Autauga County; Alabama,4121,10.0,0.6,7.6,6.2,13.4,18.1,17.7,4.0,0.2059
5,1001020502,Autauga County,Alabama,AL,Census Tract 205.02; Autauga County; Alabama,3256,13.1,5.6,5.0,0.0,13.4,30.2,11.4,6.8,0.2459
6,1001020503,Autauga County,Alabama,AL,Census Tract 205.03; Autauga County; Alabama,3513,15.7,3.5,5.7,4.1,13.5,19.7,13.1,2.5,0.1694
7,1001020600,Autauga County,Alabama,AL,Census Tract 206; Autauga County; Alabama,3839,28.1,3.2,16.5,11.3,17.1,29.7,16.1,2.5,0.4095
8,1001020700,Autauga County,Alabama,AL,Census Tract 207; Autauga County; Alabama,3369,37.1,1.2,10.3,7.0,10.8,26.7,16.7,6.5,0.6440
9,1001020801,Autauga County,Alabama,AL,Census Tract 208.01; Autauga County; Alabama,3166,12.3,3.1,7.8,7.3,20.5,20.8,15.2,5.5,0.2239



SUMMARY STATISTICS


,total_population,poverty_rate,unemployment_rate,no_hs_diploma_rate,uninsured_rate,seniors_pct,youth_pct,disability_rate,single_parent_pct,svi_percentile
count,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.00000,2000.00000,2000.000000,2000.000000,2000.000000
mean,3724.212500,26.091050,5.847350,12.388100,10.406300,17.97085,21.71805,16.158750,6.892550,0.533151
std,1689.423713,15.589443,5.075037,8.195084,6.510088,10.65714,7.45994,6.754201,5.865842,0.277174
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.00000,0.000000,0.000000,0.000000
25%,2521.750000,14.175000,2.400000,5.900000,5.900000,12.30000,17.80000,11.500000,2.900000,0.312100
50%,3582.500000,23.400000,4.600000,11.450000,9.400000,16.70000,22.00000,15.500000,5.600000,0.549850
75%,4729.000000,35.225000,8.000000,17.600000,13.900000,21.30000,26.20000,20.200000,9.400000,0.760200
max,11981.000000,93.100000,71.900000,47.700000,54.600000,100.00000,51.90000,57.200000,47.600000,0.999700


Clean CMS Hospital Data

In [5]:
# Cell 3: Clean CMS Hospital Data (CORRECTED)

def clean_cms_hospitals():
    """
    Clean and prepare CMS Hospital data
    """
    print("="*70)
    print("CLEANING CMS HOSPITAL DATA")
    print("="*70)

    # Load raw data
    print("\n Loading raw data...")
    df = pd.read_csv(f"{data_raw_path}/cms_hospitals_raw.csv")
    print(f"✓ Loaded {len(df):,} raw records")

    # Show available columns
    print(f"\n Available columns ({len(df.columns)} total):")
    for i, col in enumerate(df.columns[:20], 1):
        print(f"   {i}. {col}")
    if len(df.columns) > 20:
        print(f"   ... and {len(df.columns) - 20} more")

    # Map CMS column names to our standardized names
    # Based on the actual CMS API column names
    column_mapping = {
        'Rndrng_Prvdr_CCN': 'provider_id',
        'Rndrng_Prvdr_Org_Name': 'facility_name',
        'Rndrng_Prvdr_St': 'address',
        'Rndrng_Prvdr_City': 'city',
        'Rndrng_Prvdr_State_Abrvtn': 'state',
        'Rndrng_Prvdr_Zip5': 'zip_code',
        'Rndrng_Prvdr_RUCA_Desc': 'area_type',
        'Tot_Benes': 'total_beneficiaries',
        'Tot_Dschrgs': 'total_discharges'
    }

    # Create clean dataframe
    df_clean = pd.DataFrame()

    print("\n Mapping columns...")
    for old_name, new_name in column_mapping.items():
        if old_name in df.columns:
            df_clean[new_name] = df[old_name]
            print(f"✓ Mapped '{old_name}' → '{new_name}'")

    print(f"\n Mapped {len(df_clean.columns)} columns")

    # Data cleaning
    print("\n Cleaning data...")

    # Remove duplicates
    initial_len = len(df_clean)
    if 'facility_name' in df_clean.columns and 'city' in df_clean.columns and 'state' in df_clean.columns:
        df_clean = df_clean.drop_duplicates(subset=['facility_name', 'city', 'state'])
        print(f" Removed {initial_len - len(df_clean):,} duplicates")
    else:
        df_clean = df_clean.drop_duplicates()
        print(f" Removed {initial_len - len(df_clean):,} duplicates")

    # Clean state abbreviations
    if 'state' in df_clean.columns:
        df_clean['state'] = df_clean['state'].str.upper().str.strip()
        # Keep only valid 2-letter state codes
        df_clean = df_clean[df_clean['state'].str.len() == 2]
        print(f" Standardized state codes")

    # Clean facility names
    if 'facility_name' in df_clean.columns:
        df_clean['facility_name'] = df_clean['facility_name'].str.strip()
        print(f" Cleaned facility names")

    # Convert numeric columns
    numeric_cols = ['total_beneficiaries', 'total_discharges']
    for col in numeric_cols:
        if col in df_clean.columns:
            df_clean[col] = pd.to_numeric(df_clean[col], errors='coerce')
            print(f"✓ Converted {col} to numeric")

    # Categorize by area type (urban vs rural)
    if 'area_type' in df_clean.columns:
        print(f"\n Area type distribution:")
        area_counts = df_clean['area_type'].value_counts()
        for area, count in area_counts.head(5).items():
            print(f"   • {area}: {count:,}")

    # Remove rows with missing critical data
    critical_cols = ['facility_name', 'state']
    critical_cols = [col for col in critical_cols if col in df_clean.columns]

    if critical_cols:
        before = len(df_clean)
        df_clean = df_clean.dropna(subset=critical_cols)
        removed = before - len(df_clean)
        if removed > 0:
            print(f"\n✓ Removed {removed:,} rows with missing critical data")

    # Save cleaned data
    output_path = f"{data_processed_path}/hospitals_clean.csv"
    df_clean.to_csv(output_path, index=False)

    print(f"\n CLEANING COMPLETE!")
    print(f" Saved: {output_path}")
    print(f" Final dataset: {len(df_clean):,} providers, {len(df_clean.columns)} columns")

    return df_clean

# Run cleaning function
hospitals_df = clean_cms_hospitals()

# Display preview
print("\n" + "="*70)
print("DATA PREVIEW")
print("="*70)
display(hospitals_df.head(10))

# Show providers by state
if 'state' in hospitals_df.columns:
    print("\n" + "="*70)
    print("PROVIDERS BY STATE (Top 15)")
    print("="*70)
    state_counts = hospitals_df['state'].value_counts().head(15)
    display(state_counts.to_frame())

# Show summary statistics
if 'total_discharges' in hospitals_df.columns:
    print("\n" + "="*70)
    print("DISCHARGE STATISTICS")
    print("="*70)
    print(f"Total discharges across all providers: {hospitals_df['total_discharges'].sum():,.0f}")
    print(f"Average discharges per provider: {hospitals_df['total_discharges'].mean():,.0f}")

CLEANING CMS HOSPITAL DATA

 Loading raw data...
✓ Loaded 3,093 raw records

 Available columns (57 total):
   1. Rndrng_Prvdr_CCN
   2. Rndrng_Prvdr_Org_Name
   3. Rndrng_Prvdr_St
   4. Rndrng_Prvdr_City
   5. Rndrng_Prvdr_Zip5
   6. Rndrng_Prvdr_State_Abrvtn
   7. Rndrng_Prvdr_State_FIPS
   8. Rndrng_Prvdr_RUCA
   9. Rndrng_Prvdr_RUCA_Desc
   10. Tot_Benes
   11. Tot_Submtd_Cvrd_Chrg
   12. Tot_Pymt_Amt
   13. Tot_Mdcr_Pymt_Amt
   14. Tot_Dschrgs
   15. Tot_Cvrd_Days
   16. Tot_Days
   17. Bene_Avg_Age
   18. Bene_Age_LT_65_Cnt
   19. Bene_Age_65_74_Cnt
   20. Bene_Age_75_84_Cnt
   ... and 37 more

 Mapping columns...
✓ Mapped 'Rndrng_Prvdr_CCN' → 'provider_id'
✓ Mapped 'Rndrng_Prvdr_Org_Name' → 'facility_name'
✓ Mapped 'Rndrng_Prvdr_St' → 'address'
✓ Mapped 'Rndrng_Prvdr_City' → 'city'
✓ Mapped 'Rndrng_Prvdr_State_Abrvtn' → 'state'
✓ Mapped 'Rndrng_Prvdr_Zip5' → 'zip_code'
✓ Mapped 'Rndrng_Prvdr_RUCA_Desc' → 'area_type'
✓ Mapped 'Tot_Benes' → 'total_beneficiaries'
✓ Mapped 'Tot_Dsch

,provider_id,facility_name,address,city,state,zip_code,area_type,total_beneficiaries,total_discharges
0,10001,Southeast Health Medical Center,1108 Ross Clark Circle,Dothan,AL,36301,Metropolitan area high commuting: primary flow...,3088,4059
1,10005,Marshall Medical Centers South Campus,2505 U S Highway 431 North,Boaz,AL,35957,Micropolitan area core: primary flow within an...,1123,1410
2,10006,North Alabama Medical Center,1701 Veterans Drive,Florence,AL,35630,Metropolitan area core: primary flow within an...,2634,3621
3,10007,Mizell Memorial Hospital,702 N Main St,Opp,AL,36467,Small town core: primary flow within an urban ...,252,353
4,10008,Crenshaw Community Hospital,101 Hospital Circle,Luverne,AL,36049,Metropolitan area low commuting: primary flow ...,89,123
5,10011,St Vincent's East,50 Medical Park East Drive,Birmingham,AL,35235,Metropolitan area core: primary flow within an...,1943,2609
6,10012,Dekalb Regional Medical Center,200 Med Center Drive,Fort Payne,AL,35968,Rural areas: primary flow to a tract outside a...,454,571
7,10016,Shelby Baptist Medical Center,1000 First Street North,Alabaster,AL,35007,Metropolitan area core: primary flow within an...,1143,1504
8,10018,Uab Callahan Eye Hospital Authority,1720 University Blvd Ste 305,Birmingham,AL,35233,Metropolitan area core: primary flow within an...,11,11
9,10019,Helen Keller Memorial Hospital,1300 South Montgomery Avenue,Sheffield,AL,35660,Metropolitan area core: primary flow within an...,1152,1535



PROVIDERS BY STATE (Top 15)


,count
state,
CA,279
TX,278
FL,168
PA,134
NY,132
OH,119
IL,118
GA,95
MI,90



DISCHARGE STATISTICS
Total discharges across all providers: 6,920,995
Average discharges per provider: 2,238


Create Combined Analytics Dataset with Calculated Metrics

In [8]:
def create_analytics_dataset(vulnerability_df, hospitals_df):
    """
    Combine datasets and calculate vulnerability scores
    """
    print("="*70)
    print("CREATING COMBINED ANALYTICS DATASET")
    print("="*70)

    # Count hospitals per state
    print("\n Calculating hospital counts by state...")
    hospital_counts = hospitals_df.groupby('state').size().reset_index()
    hospital_counts.columns = ['state', 'hospital_count']

    print(f"✓ Calculated hospital counts for {len(hospital_counts)} states")
    print("\nTop 10 states by hospital count:")
    display(hospital_counts.nlargest(10, 'hospital_count'))

    # Merge with vulnerability data
    print("\n Merging datasets...")
    combined = vulnerability_df.merge(hospital_counts, on='state', how='left')
    combined['hospital_count'] = combined['hospital_count'].fillna(0)
    print(f"✓ Merged data: {len(combined):,} counties")

    # Calculate state-level metrics
    print("\n Calculating state-level metrics...")
    state_pop = combined.groupby('state')['total_population'].sum().reset_index()
    state_pop.columns = ['state', 'state_population']

    combined = combined.merge(state_pop, on='state', how='left')

    # Hospitals per 100k population
    combined['hospitals_per_100k'] = (
        (combined['hospital_count'] / combined['state_population']) * 100000
    ).round(2)

    print(" Calculated hospitals per 100k population")

    # CREATE COMPOSITE VULNERABILITY SCORE
    print("\n Creating composite vulnerability score...")

    # Weights for different factors (total = 1.0)
    weights = {
        'poverty_rate': 0.25,          # 25%
        'uninsured_rate': 0.25,        # 25%
        'seniors_pct': 0.20,           # 20%
        'disability_rate': 0.15,       # 15%
        'unemployment_rate': 0.15      # 15%
    }

    # Calculate weighted score
    combined['vulnerability_score'] = 0

    for factor, weight in weights.items():
        if factor in combined.columns:
            combined['vulnerability_score'] += combined[factor] * weight
            print(f"✓ Added {factor} (weight: {weight})")

    # Normalize to 0-100 scale
    max_score = combined['vulnerability_score'].max()
    if max_score > 0:
        combined['vulnerability_score'] = (
            combined['vulnerability_score'] / max_score * 100
        ).round(2)

    print(f" Vulnerability score calculated (0-100 scale)")
    print(f"   • Mean: {combined['vulnerability_score'].mean():.2f}")
    print(f"   • Median: {combined['vulnerability_score'].median():.2f}")
    print(f"   • Max: {combined['vulnerability_score'].max():.2f}")

    # CREATE RISK CATEGORIES
    print("\n  Creating risk categories...")
    combined['risk_category'] = pd.cut(
        combined['vulnerability_score'],
        bins=[0, 33, 66, 100],
        labels=['Low Risk', 'Moderate Risk', 'High Risk'],
        include_lowest=True
    )

    print(" Risk categories assigned:")
    print(combined['risk_category'].value_counts())

    # CREATE ACCESS CATEGORIES
    print("\n Categorizing healthcare access...")

    conditions = [
        # Healthcare desert: Low hospital access + significant population
        (combined['hospitals_per_100k'] < 2) & (combined['total_population'] > 20000),
        # Limited access: Moderate hospital access
        (combined['hospitals_per_100k'] < 5) & (combined['total_population'] > 10000)
    ]

    choices = ['Healthcare Desert', 'Limited Access']

    combined['access_category'] = np.select(conditions, choices, default='Adequate Access')

    print(" Access categories assigned:")
    print(combined['access_category'].value_counts())

    # IDENTIFY PRIORITY COUNTIES
    print("\n Identifying priority intervention counties...")

    # High risk + poor access = highest priority
    combined['priority_score'] = 0

    # Add points for high vulnerability
    combined.loc[combined['risk_category'] == 'High Risk', 'priority_score'] += 3
    combined.loc[combined['risk_category'] == 'Moderate Risk', 'priority_score'] += 2
    combined.loc[combined['risk_category'] == 'Low Risk', 'priority_score'] += 1

    # Add points for poor access
    combined.loc[combined['access_category'] == 'Healthcare Desert', 'priority_score'] += 3
    combined.loc[combined['access_category'] == 'Limited Access', 'priority_score'] += 2

    # Add points for large vulnerable populations
    combined.loc[combined['total_population'] > 50000, 'priority_score'] += 1

    # Create priority categories
    combined['priority_level'] = pd.cut(
        combined['priority_score'],
        bins=[0, 3, 5, 10],
        labels=['Low Priority', 'Medium Priority', 'High Priority'],
        include_lowest=True
    )

    print("✓ Priority levels assigned:")
    print(combined['priority_level'].value_counts())

    # Calculate affected population
    print("\n Calculating affected populations...")

    high_risk_pop = combined[combined['risk_category'] == 'High Risk']['total_population'].sum()
    desert_pop = combined[combined['access_category'] == 'Healthcare Desert']['total_population'].sum()
    high_priority_pop = combined[combined['priority_level'] == 'High Priority']['total_population'].sum()

    print(f"   • High vulnerability counties: {high_risk_pop:,} people")
    print(f"   • Healthcare deserts: {desert_pop:,} people")
    print(f"   • High priority areas: {high_priority_pop:,} people")

    # Save combined dataset
    output_path = f"{data_processed_path}/combined_analytics.csv"
    combined.to_csv(output_path, index=False)

    print(f"\n ANALYTICS DATASET COMPLETE!")
    print(f" Saved: {output_path}")
    print(f" Final dataset: {len(combined):,} counties, {len(combined.columns)} columns")

    return combined

# Run the function
combined_df = create_analytics_dataset(vulnerability_df, hospitals_df)

# Display preview
print("\n" + "="*70)
print("COMBINED DATASET PREVIEW")
print("="*70)
display(combined_df.head(10))

# Show key columns
key_cols = ['county', 'state', 'total_population', 'vulnerability_score',
            'hospitals_per_100k', 'risk_category', 'access_category', 'priority_level']
key_cols = [col for col in key_cols if col in combined_df.columns]

print("\n" + "="*70)
print("KEY METRICS PREVIEW")
print("="*70)
display(combined_df[key_cols].head(15))

CREATING COMBINED ANALYTICS DATASET

 Calculating hospital counts by state...
✓ Calculated hospital counts for 51 states

Top 10 states by hospital count:


,state,hospital_count
4,CA,279
43,TX,278
9,FL,168
38,PA,134
34,NY,132
35,OH,119
14,IL,118
10,GA,95
22,MI,90
27,NC,84



 Merging datasets...
✓ Merged data: 2,000 counties

 Calculating state-level metrics...
 Calculated hospitals per 100k population

 Creating composite vulnerability score...
✓ Added poverty_rate (weight: 0.25)
✓ Added uninsured_rate (weight: 0.25)
✓ Added seniors_pct (weight: 0.2)
✓ Added disability_rate (weight: 0.15)
✓ Added unemployment_rate (weight: 0.15)
 Vulnerability score calculated (0-100 scale)
   • Mean: 28.92
   • Median: 28.12
   • Max: 100.00

  Creating risk categories...
 Risk categories assigned:
risk_category
Low Risk         1310
Moderate Risk     689
High Risk           1
Name: count, dtype: int64

 Categorizing healthcare access...
 Access categories assigned:
access_category
Adequate Access    1993
Limited Access        7
Name: count, dtype: int64

 Identifying priority intervention counties...
✓ Priority levels assigned:
priority_level
Low Priority       1997
Medium Priority       3
High Priority         0
Name: count, dtype: int64

 Calculating affected populat

,fips,county,state,state_abbr,location,total_population,poverty_rate,unemployment_rate,no_hs_diploma_rate,uninsured_rate,...,single_parent_pct,svi_percentile,hospital_count,state_population,hospitals_per_100k,vulnerability_score,risk_category,access_category,priority_score,priority_level
0,1001020100,Autauga County,Alabama,AL,Census Tract 201; Autauga County; Alabama,1865,21.6,2.6,16.2,8.2,...,4.9,0.3635,0.0,5028092,0.0,26.64,Low Risk,Adequate Access,1,Low Priority
1,1001020200,Autauga County,Alabama,AL,Census Tract 202; Autauga County; Alabama,1861,14.3,5.5,8.7,7.8,...,0.7,0.4155,0.0,5028092,0.0,22.66,Low Risk,Adequate Access,1,Low Priority
2,1001020300,Autauga County,Alabama,AL,Census Tract 203; Autauga County; Alabama,3492,22.2,1.9,10.6,5.1,...,8.5,0.4843,0.0,5028092,0.0,22.04,Low Risk,Adequate Access,1,Low Priority
3,1001020400,Autauga County,Alabama,AL,Census Tract 204; Autauga County; Alabama,3987,12.9,0.9,7.3,4.4,...,3.7,0.2386,0.0,5028092,0.0,23.48,Low Risk,Adequate Access,1,Low Priority
4,1001020501,Autauga County,Alabama,AL,Census Tract 205.01; Autauga County; Alabama,4121,10.0,0.6,7.6,6.2,...,4.0,0.2059,0.0,5028092,0.0,17.11,Low Risk,Adequate Access,1,Low Priority
5,1001020502,Autauga County,Alabama,AL,Census Tract 205.02; Autauga County; Alabama,3256,13.1,5.6,5.0,0.0,...,6.8,0.2459,0.0,5028092,0.0,15.35,Low Risk,Adequate Access,1,Low Priority
6,1001020503,Autauga County,Alabama,AL,Census Tract 205.03; Autauga County; Alabama,3513,15.7,3.5,5.7,4.1,...,2.5,0.1694,0.0,5028092,0.0,18.31,Low Risk,Adequate Access,1,Low Priority
7,1001020600,Autauga County,Alabama,AL,Census Tract 206; Autauga County; Alabama,3839,28.1,3.2,16.5,11.3,...,2.5,0.4095,0.0,5028092,0.0,29.18,Low Risk,Adequate Access,1,Low Priority
8,1001020700,Autauga County,Alabama,AL,Census Tract 207; Autauga County; Alabama,3369,37.1,1.2,10.3,7.0,...,6.5,0.6440,0.0,5028092,0.0,28.65,Low Risk,Adequate Access,1,Low Priority
9,1001020801,Autauga County,Alabama,AL,Census Tract 208.01; Autauga County; Alabama,3166,12.3,3.1,7.8,7.3,...,5.5,0.2239,0.0,5028092,0.0,21.20,Low Risk,Adequate Access,1,Low Priority



KEY METRICS PREVIEW


,county,state,total_population,vulnerability_score,hospitals_per_100k,risk_category,access_category,priority_level
0,Autauga County,Alabama,1865,26.64,0.0,Low Risk,Adequate Access,Low Priority
1,Autauga County,Alabama,1861,22.66,0.0,Low Risk,Adequate Access,Low Priority
2,Autauga County,Alabama,3492,22.04,0.0,Low Risk,Adequate Access,Low Priority
3,Autauga County,Alabama,3987,23.48,0.0,Low Risk,Adequate Access,Low Priority
4,Autauga County,Alabama,4121,17.11,0.0,Low Risk,Adequate Access,Low Priority
5,Autauga County,Alabama,3256,15.35,0.0,Low Risk,Adequate Access,Low Priority
6,Autauga County,Alabama,3513,18.31,0.0,Low Risk,Adequate Access,Low Priority
7,Autauga County,Alabama,3839,29.18,0.0,Low Risk,Adequate Access,Low Priority
8,Autauga County,Alabama,3369,28.65,0.0,Low Risk,Adequate Access,Low Priority
9,Autauga County,Alabama,3166,21.20,0.0,Low Risk,Adequate Access,Low Priority


Generate Comprehensive Data Quality Report


In [9]:
def generate_final_report(combined_df):
    """
    Generate comprehensive data quality and insights report
    """
    print("="*70)
    print(" FINAL DATA QUALITY & INSIGHTS REPORT")
    print("="*70)
    print(f"Generated: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

    # Dataset Overview
    print("\n" + "="*70)
    print("1. DATASET OVERVIEW")
    print("="*70)
    print(f"   • Total Counties: {len(combined_df):,}")
    print(f"   • States Covered: {combined_df['state'].nunique()}")
    print(f"   • Total Population: {combined_df['total_population'].sum():,}")
    print(f"   • Data Columns: {len(combined_df.columns)}")

    # Vulnerability Statistics
    print("\n" + "="*70)
    print("2. VULNERABILITY ANALYSIS")
    print("="*70)

    if 'vulnerability_score' in combined_df.columns:
        print(f"   Vulnerability Score Statistics:")
        print(f"   • Mean: {combined_df['vulnerability_score'].mean():.2f}")
        print(f"   • Median: {combined_df['vulnerability_score'].median():.2f}")
        print(f"   • Std Dev: {combined_df['vulnerability_score'].std():.2f}")
        print(f"   • Min: {combined_df['vulnerability_score'].min():.2f}")
        print(f"   • Max: {combined_df['vulnerability_score'].max():.2f}")

    if 'risk_category' in combined_df.columns:
        print(f"\n   Risk Category Distribution:")
        risk_dist = combined_df['risk_category'].value_counts()
        for category, count in risk_dist.items():
            pct = (count / len(combined_df)) * 100
            print(f"   • {category}: {count:,} counties ({pct:.1f}%)")

    # Hospital Access Analysis
    print("\n" + "="*70)
    print("3. HOSPITAL ACCESS ANALYSIS")
    print("="*70)

    if 'hospitals_per_100k' in combined_df.columns:
        print(f"   Hospitals per 100k Population:")
        print(f"   • Mean: {combined_df['hospitals_per_100k'].mean():.2f}")
        print(f"   • Median: {combined_df['hospitals_per_100k'].median():.2f}")

    if 'access_category' in combined_df.columns:
        print(f"\n   Access Category Distribution:")
        access_dist = combined_df['access_category'].value_counts()
        for category, count in access_dist.items():
            pct = (count / len(combined_df)) * 100
            pop = combined_df[combined_df['access_category'] == category]['total_population'].sum()
            print(f"   • {category}: {count:,} counties ({pct:.1f}%) - {pop:,} people")

    # Priority Analysis
    print("\n" + "="*70)
    print("4. PRIORITY INTERVENTION AREAS")
    print("="*70)

    if 'priority_level' in combined_df.columns:
        priority_dist = combined_df['priority_level'].value_counts()
        for level, count in priority_dist.items():
            pop = combined_df[combined_df['priority_level'] == level]['total_population'].sum()
            print(f"   • {level}: {count:,} counties - {pop:,} people")

    # Top 10 Most Vulnerable Counties
    print("\n" + "="*70)
    print("5. TOP 10 MOST VULNERABLE COUNTIES")
    print("="*70)

    if 'vulnerability_score' in combined_df.columns:
        top_10 = combined_df.nlargest(10, 'vulnerability_score')
        display_cols = ['county', 'state', 'total_population', 'vulnerability_score',
                       'hospitals_per_100k', 'risk_category', 'access_category']
        display_cols = [col for col in display_cols if col in top_10.columns]
        display(top_10[display_cols])

    # Top 10 Healthcare Deserts
    print("\n" + "="*70)
    print("6. TOP 10 LARGEST HEALTHCARE DESERTS (by population)")
    print("="*70)

    if 'access_category' in combined_df.columns:
        deserts = combined_df[combined_df['access_category'] == 'Healthcare Desert']
        if len(deserts) > 0:
            top_deserts = deserts.nlargest(10, 'total_population')
            display_cols = ['county', 'state', 'total_population', 'vulnerability_score',
                           'hospitals_per_100k']
            display_cols = [col for col in display_cols if col in top_deserts.columns]
            display(top_deserts[display_cols])
        else:
            print("   No healthcare deserts identified with current criteria")

    # Data Quality Metrics
    print("\n" + "="*70)
    print("7. DATA QUALITY METRICS")
    print("="*70)

    print(f"   Missing Values by Column:")
    missing = combined_df.isnull().sum()
    missing = missing[missing > 0].sort_values(ascending=False)

    if len(missing) > 0:
        for col, count in missing.items():
            pct = (count / len(combined_df)) * 100
            print(f"   • {col}: {count:,} ({pct:.1f}%)")
    else:
        print("   ✓ No missing values!")

    print(f"\n   • Duplicate FIPS codes: {combined_df['fips'].duplicated().sum()}")
    print(f"   • Records with complete data: {len(combined_df.dropna()):,}")

    # Save report to text file
    report_path = f"{data_processed_path}/analysis_report.txt"

    with open(report_path, 'w') as f:
        f.write("HEALTHCARE EQUITY ANALYSIS - DATA REPORT\n")
        f.write("=" * 70 + "\n")
        f.write(f"Generated: {datetime.now()}\n\n")
        f.write(f"Total Counties Analyzed: {len(combined_df):,}\n")
        f.write(f"Total Population Covered: {combined_df['total_population'].sum():,}\n")

        if 'access_category' in combined_df.columns:
            deserts = len(combined_df[combined_df['access_category'] == 'Healthcare Desert'])
            f.write(f"Healthcare Deserts Identified: {deserts}\n")

        f.write("\nKey Findings:\n")
        f.write("- Dataset contains comprehensive vulnerability and access metrics\n")
        f.write("- Priority counties identified for intervention\n")
        f.write("- Ready for Tableau visualization\n")

    print(f"\n Report saved to: {report_path}")
    print("\n ANALYSIS COMPLETE! Ready for Tableau visualization.")

# Generate the final report
generate_final_report(combined_df)

# Show final column list
print("\n" + "="*70)
print("AVAILABLE COLUMNS FOR TABLEAU")
print("="*70)
for i, col in enumerate(combined_df.columns, 1):
    print(f"{i:2}. {col}")

 FINAL DATA QUALITY & INSIGHTS REPORT
Generated: 2026-02-09 20:36:15

1. DATASET OVERVIEW
   • Total Counties: 2,000
   • States Covered: 3
   • Total Population: 7,448,425
   • Data Columns: 23

2. VULNERABILITY ANALYSIS
   Vulnerability Score Statistics:
   • Mean: 28.92
   • Median: 28.12
   • Std Dev: 10.72
   • Min: 0.00
   • Max: 100.00

   Risk Category Distribution:
   • Low Risk: 1,310 counties (65.5%)
   • Moderate Risk: 689 counties (34.4%)
   • High Risk: 1 counties (0.1%)

3. HOSPITAL ACCESS ANALYSIS
   Hospitals per 100k Population:
   • Mean: 0.00
   • Median: 0.00

   Access Category Distribution:
   • Adequate Access: 1,993 counties (99.7%) - 7,371,413 people
   • Limited Access: 7 counties (0.4%) - 77,012 people

4. PRIORITY INTERVENTION AREAS
   • Low Priority: 1,997 counties - 7,415,762 people
   • Medium Priority: 3 counties - 32,663 people
   • High Priority: 0 counties - 0 people

5. TOP 10 MOST VULNERABLE COUNTIES


,county,state,total_population,vulnerability_score,hospitals_per_100k,risk_category,access_category
1741,La Paz County,Arizona,916,100.00,0.0,High Risk,Adequate Access
492,Jefferson County,Alabama,2099,61.09,0.0,Moderate Risk,Adequate Access
295,Dallas County,Alabama,2524,60.81,0.0,Moderate Risk,Adequate Access
999,Mobile County,Alabama,1057,60.73,0.0,Moderate Risk,Adequate Access
506,Jefferson County,Alabama,2609,60.53,0.0,Moderate Risk,Adequate Access
483,Jefferson County,Alabama,4121,59.59,0.0,Moderate Risk,Adequate Access
445,Houston County,Alabama,2296,59.01,0.0,Moderate Risk,Adequate Access
109,Calhoun County,Alabama,1019,58.82,0.0,Moderate Risk,Adequate Access
1613,Apache County,Arizona,1549,58.40,0.0,Moderate Risk,Adequate Access
1742,La Paz County,Arizona,790,58.40,0.0,Moderate Risk,Adequate Access



6. TOP 10 LARGEST HEALTHCARE DESERTS (by population)
   No healthcare deserts identified with current criteria

7. DATA QUALITY METRICS
   Missing Values by Column:
   ✓ No missing values!

   • Duplicate FIPS codes: 0
   • Records with complete data: 2,000

 Report saved to: /content/drive/MyDrive/Healthcare_Analysis_Project/data_processed/analysis_report.txt

 ANALYSIS COMPLETE! Ready for Tableau visualization.

AVAILABLE COLUMNS FOR TABLEAU
 1. fips
 2. county
 3. state
 4. state_abbr
 5. location
 6. total_population
 7. poverty_rate
 8. unemployment_rate
 9. no_hs_diploma_rate
10. uninsured_rate
11. seniors_pct
12. youth_pct
13. disability_rate
14. single_parent_pct
15. svi_percentile
16. hospital_count
17. state_population
18. hospitals_per_100k
19. vulnerability_score
20. risk_category
21. access_category
22. priority_score
23. priority_level
